### Analyze ECoG data from: https://doi.org/10.5281/zenodo.16954184
author: JTM

In [1]:
# imports
# python standard library
from warnings import warn
from functools import partial
from itertools import starmap

# dependencies
# fooof - MY FORK of lorentzian branch
# https://github.com/jtmiles/fooof/tree/lorentzian/fooof/
from fooof import FOOOF, FOOOFGroup
from fooof.core.funcs import lorentzian_function
from fooof.utils import interpolate_spectra
from fooof.utils.params import compute_time_constant, compute_knee_frequency
from fooof.sim.gen import gen_periodic #, gen_freqs, gen_aperiodic, gen_model 

# standard scientific packages
import numpy as np
import pandas as pd
from scipy.ndimage import uniform_filter as unifilt
from scipy import interpolate
from scipy.optimize import curve_fit

# from my repo
# https://github.com/jtmiles/PAF_INT_neurodev
from PAF_INT_neurodev.iEEG_utils.processing.bumps import get_bump_ixs
from PAF_INT_neurodev.iEEG_utils.processing.fooof_mod import fit_group

# CHANGE BASED ON LOCAL FILE STORAGE
# expecting a csv, generated by load_ephys_save_spect
f_path = r"ADD DATAPATH HERE" 
spect_table = pd.read_csv(f_path+r"\restingstate_spectra.csv")

C:\Users\jmile3\AppData\Local\Temp\ipykernel_25368\3979284465.py:10: DeprecationWarning: 
The `fooof` package is being deprecated and replaced by the `specparam` (spectral parameterization) package.
This version of `fooof` (1.1) is fully functional, but will not be further updated.
New projects are recommended to update to using `specparam` (see Changelog for details).
  from fooof import FOOOF, FOOOFGroup
C:\Users\jmile3\AppData\Local\Temp\ipykernel_25368\3979284465.py:30: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  spect_table = pd.read_csv(f_path+r"\restingstate_spectra.csv")


In [2]:
# Frequencies were saved as column names in a table
fcols = spect_table.columns.str.contains(r'\d')
freqs = np.array(spect_table.columns[fcols],dtype='float64')
# The current data set doesn't keep spectral data below 0.5 Hz, so be wary
if freqs[0] != 0.5:
    warn("check column names!")
else:
    # separate spectral data into an array
    spectra = spect_table.iloc[:,fcols].to_numpy()
    # do some light smoothing of adjacent frequencies and timepoints
    # (rows are temporally adjacent with 0.25 sec offset. should technically split by region/id)
    smoothspect = unifilt(spectra, [5,3], mode="nearest",)
    # interpolate over 60 Hz line noise to make fitting better
    _,spectra = interpolate_spectra(freqs, smoothspect, [58.5,61.5], buffer=5)

In [4]:
def model_spect(index,model_obj,freqs,min_f,max_f,height=0.1,intfs=intfs):
    '''
    index = int (pass to starmap as iterable of ints for multiprocessing)
    model_obj = fooof model object that has already been initialized
    freqs = iterable of frequencies to fit and pass to model_obj
    min_f = minimum frequency to search for peaks in
    max_f = maximum frequency to search for peaks in

    returns a list with:
        peak frequency between min_f and max_f
        amplitude of peak frequency
        INT (from knee frequency)
        offset of aperiodic model
        knee frequency of aperiodic model
        exponent of aperiodic model
    ^^^ might be good to return as df or dict?
    
    ''' 
    # refit the original peak-removed spectrum with reweighted low and high frequency spacing
    pk_rm = model_obj.power_spectra[index]-gen_periodic(freqs,np.ndarray.flatten(model_obj.group_results[index].gaussian_params))
    intfx = interpolate.interp1d(freqs,10**pk_rm)
    offset, knee, exp = model_obj.get_params("aperiodic_params")[index,:]
    try:
        reparams,_ = curve_fit(lorentzian_function, intfs, np.log10(intfx(intfs)), p0=[offset,knee,exp], maxfev=2500)
    except:
        reparams = [offset, knee, exp]
        print("!")
    offset,knee,exp = reparams
    # whitened spectrum (lorentzian corrected)
    resid_trace = model_obj.power_spectra[index]-lorentzian_function(freqs, offset, knee, exp)
    # save the fit at f_min (useful for filtering bad model fits)
    fit0 = resid_trace[0]
    # find peak alpha frequency
    # only keep max value frequency bump in range, with required resid pow >= "height"
    # width is required (min,max) base (lowest contour, i.e. rel_height = 1.0) width
    bmp_ixs = get_bump_ixs(min_f,max_f,resid_trace,keep_max=True,freqs=freqs,height=height,
                           prominence=0.066,width=(3,16),rel_height=1.0)
    if bmp_ixs>0:
        PAF = freqs[bmp_ixs]
        PAA = resid_trace[bmp_ixs]
    else:
        PAF = np.nan
        PAA = np.nan
    # convert knee to INT (in milliseconds, using unlogged knee frequency)
    INT = 1000*compute_time_constant(10**(compute_knee_frequency(knee, exp)))
    return [fit0,PAF,PAA,INT,offset,knee,exp]
    

In [5]:
%%time
# do spectral parameterization model fitting

participants = spect_table.ID.unique()
info_cols = ["ID","age","region","time"]
alldfs = []
# fit across "reweighted" frequency spacing
# intended to redistribute fitting error more evenly 
# across frequency orders of magnitude (<1, 1-10, >10 Hz)
intfs = np.concatenate((np.linspace(0.5,1,sum(freqs>50)+1),np.linspace(1.05,10,sum(freqs>10)),freqs[freqs>10]))

# the multithreading seems to work better when broken down into a certain sized chunk,
# so going to try breaking down by ID to hopefully speed up
for ID in participants:
    print(ID)
    s_df = spect_table.loc[spect_table.ID==ID,:]
    # subject indices from original dataframe
    sub_ixs = s_df.index
    # parameterized spectra
    sub_models = fit_group(freqs,spectra[sub_ixs,:])
    # have to explicity make an iterable of single element iterables
    # (only single changing element in partial function, as set up)
    rowixs = [[ix] for ix in range(len(sub_ixs))]
    # partial function across all spectra
    partial_func = partial(model_spect,model_obj=sub_models,
                           freqs=freqs,intfs=intfs,min_f=2,max_f=14,height=0.15)
    # applying row-wise instead of with for loop on the model object
    sub_fits = starmap(partial_func,rowixs)
    # create df out of aperiodic and rhythm info
    subdf = pd.DataFrame(sub_fits,columns=["fit0","PAF","PAA","INT","offset","knee","exp"])
    # add identifying information (have to reset indices to add to the recently computed df)
    subdf[info_cols] = spect_table.loc[sub_ixs,info_cols].reset_index(drop=True)
    # stupid, but reassign original indices
    subdf.index=sub_ixs
    # will concatenate to form single df after the fact (could just append right away...)
    alldfs.append(subdf)


86b2be
Running FOOOFGroup across 5406 power spectra.


C:\Users\jmile3\python\envs\iEEG_analysis\Lib\site-packages\fooof_lorentzian\fooof\fooof\utils\params.py:24: RuntimeWarning: invalid value encountered in scalar power
  return knee ** (1./exponent)
C:\Users\jmile3\python\envs\iEEG_analysis\Lib\site-packages\fooof_lorentzian\fooof\fooof\utils\params.py:24: RuntimeWarning: invalid value encountered in scalar power
  return knee ** (1./exponent)
C:\Users\jmile3\python\envs\iEEG_analysis\Lib\site-packages\fooof_lorentzian\fooof\fooof\utils\params.py:24: RuntimeWarning: invalid value encountered in scalar power
  return knee ** (1./exponent)
C:\Users\jmile3\python\envs\iEEG_analysis\Lib\site-packages\fooof_lorentzian\fooof\fooof\utils\params.py:24: RuntimeWarning: invalid value encountered in scalar power
  return knee ** (1./exponent)
C:\Users\jmile3\python\envs\iEEG_analysis\Lib\site-packages\fooof_lorentzian\fooof\fooof\utils\params.py:24: RuntimeWarning: invalid value encountered in scalar power
  return knee ** (1./exponent)
C:\Users\j

d419f2
Running FOOOFGroup across 9605 power spectra.


C:\Users\jmile3\python\envs\iEEG_analysis\Lib\site-packages\fooof_lorentzian\fooof\fooof\utils\params.py:24: RuntimeWarning: invalid value encountered in scalar power
  return knee ** (1./exponent)
C:\Users\jmile3\python\envs\iEEG_analysis\Lib\site-packages\fooof_lorentzian\fooof\fooof\utils\params.py:24: RuntimeWarning: invalid value encountered in scalar power
  return knee ** (1./exponent)
C:\Users\jmile3\python\envs\iEEG_analysis\Lib\site-packages\fooof_lorentzian\fooof\fooof\utils\params.py:24: RuntimeWarning: invalid value encountered in scalar power
  return knee ** (1./exponent)
C:\Users\jmile3\python\envs\iEEG_analysis\Lib\site-packages\fooof_lorentzian\fooof\fooof\utils\params.py:24: RuntimeWarning: invalid value encountered in scalar power
  return knee ** (1./exponent)
C:\Users\jmile3\python\envs\iEEG_analysis\Lib\site-packages\fooof_lorentzian\fooof\fooof\utils\params.py:24: RuntimeWarning: invalid value encountered in scalar power
  return knee ** (1./exponent)
C:\Users\j

4ac2b6
Running FOOOFGroup across 11796 power spectra.
71944e
Running FOOOFGroup across 5844 power spectra.


...

fca96e
Running FOOOFGroup across 8184 power spectra.


C:\Users\jmile3\python\envs\iEEG_analysis\Lib\site-packages\fooof_lorentzian\fooof\fooof\utils\params.py:24: RuntimeWarning: invalid value encountered in scalar power
  return knee ** (1./exponent)


ecb43e
Running FOOOFGroup across 14382 power spectra.


C:\Users\jmile3\python\envs\iEEG_analysis\Lib\site-packages\fooof_lorentzian\fooof\fooof\utils\params.py:24: RuntimeWarning: invalid value encountered in scalar power
  return knee ** (1./exponent)
C:\Users\jmile3\python\envs\iEEG_analysis\Lib\site-packages\fooof_lorentzian\fooof\fooof\utils\params.py:24: RuntimeWarning: invalid value encountered in scalar power
  return knee ** (1./exponent)
C:\Users\jmile3\python\envs\iEEG_analysis\Lib\site-packages\fooof_lorentzian\fooof\fooof\utils\params.py:24: RuntimeWarning: invalid value encountered in scalar power
  return knee ** (1./exponent)
C:\Users\jmile3\python\envs\iEEG_analysis\Lib\site-packages\fooof_lorentzian\fooof\fooof\utils\params.py:24: RuntimeWarning: invalid value encountered in scalar power
  return knee ** (1./exponent)


CPU times: total: 1h 31min 18s
Wall time: 1h 51min 16s


In [6]:
params_df = pd.concat(alldfs).reindex(columns=np.roll(subdf.columns,4)) # move columns around
params_df.to_csv(f_path+r"\params_table_TEST.csv",index=False)